# I- DESCRIPTION

In [ ]:
qMRinfo('qmt_spgr'); % Describe the model

# II- MODEL PARAMETERS
## a- Create object

In [ ]:
Model = qmt_spgr;

## Download sample data from OSF
> The current `Model` is an instance of `qmt_spgr` class.
 
You can manually download the sample data for `qmt_spgr` [by clicking here](https://osf.io/pzqyn/download?version=2).

In [ ]:
dataDir = downloadData(Model,pwd);

## b- Set protocol
> Protocol is set according to the example data
 
%          |- qmt_spgr object needs 2 protocol field(s) to be assigned:
%          |-   MTdata
%          |-   TimingTable

In [ ]:
Angle = [142.0000; 426.0000; 142.0000; 426.0000; 142.0000; 426.0000; 142.0000; 426.0000; 142.0000; 426.0000];
% Angle is a vector of [10X1]
Offset = [443.0000; 443.0000; 1088.0000; 1088.0000; 2732.0000; 2732.0000; 6862.0000; 6862.0000; 17235.0000; 17235.0000];
% Offset is a vector of [10X1]
Model.Prot.MTdata.Mat = [ Angle Offset];
%%   
Tmt  = 0.0102;
Ts  = 0.003;
Tp  = 0.0018;
Tr  = 0.01;
TR  = 0.025;
Model.Prot.TimingTable.Mat = [ Tmt  Ts  Tp  Tr  TR ];
%%   

# III- FIT EXPERIMENTAL DATASET
## a- Load experimental data
%          |- qmt_spgr object needs 5 data input(s) to be assigned:
%          |-   MTdata
%          |-   R1map
%          |-   B1map
%          |-   B0map
%          |-   Mask

In [ ]:
 
% MTdata.mat contains [88  128    1   10] data.
 load('qmt_spgr_data/MTdata.mat');
% R1map.mat contains [88  128] data.
 load('qmt_spgr_data/R1map.mat');
% B1map.mat contains [88  128] data.
 load('qmt_spgr_data/B1map.mat');
% B0map.mat contains [88  128] data.
 load('qmt_spgr_data/B0map.mat');
% Mask.mat contains [88  128] data.
 load('qmt_spgr_data/Mask.mat');
 data.MTdata= double(MTdata);
 data.R1map= double(R1map);
 data.B1map= double(B1map);
 data.B0map= double(B0map);
 data.Mask= double(Mask);

## b- Fit experimental data
> This section will fit data.

In [ ]:
FitResults = FitData(data,Model,0);

## c- Show fitting results
> * Output map will be displayed.
 
> * If available, a graph will be displayed to show fitting in a voxel.

In [ ]:
qMRshowOutput(FitResults,data,Model);

## d- Save results
> * qMR maps are saved in NIFTI and in a structure `FitResults.mat` that can be loaded in qMRLab graphical user interface.
> * Model object stores all the options and protocol
> * These objects can be easily shared or be used for simulation.

In [ ]:
FitResultsSave_nii(FitResults);

# IV- SIMULATIONS
> This section can be executed to run simulations for vfa_t1.
 
## a- Single Voxel Curve
> Simulates single voxel curves:
 
       1. Use equation to generate synthetic MRI data
       2. Add rician noise
       3. Fit and plot curve

In [ ]:
      x = struct;
      x.F = 0.16;
      x.kr = 30;
      x.R1f = 1;
      x.R1r = 1;
      x.T2f = 0.03;
      x.T2r = 1.3e-05;
      % Set simulation options
      Opt.SNR = 50;
      Opt.Method = 'Analytical equation';
      Opt.ResetMz = false;
      % run simulation
      figure('Name','Single Voxel Curve Simulation');
      FitResult = Model.Sim_Single_Voxel_Curve(x,Opt);

## b- Sensitivity analysis
> Simulates sensitivity to fitted parameters: 
 
       1. Vary fitting parameters from lower (lb) to upper (ub) bound.
       2. Run Sim_Single_Voxel_Curve Nofruns times
       3. Compute mean and std across runs

In [ ]:
      %              F             kr            R1f           R1r           T2f           T2r           
      OptTable.st = [0.16          30            1             1             0.03          1.3e-05]; % nominal values
      OptTable.fx = [0             1             1             1             1             1]; %vary F...
      OptTable.lb = [0.0001        0.0001        0.05          0.05          0.003         3e-06]; %...from 0.0001
      OptTable.ub = [0.5           1e+02         5             5             0.5           5e-05]; %...to 0.5
      % Set simulation options
      Opt.SNR = 50;
      Opt.Method = 'Analytical equation';
      Opt.ResetMz = false;
      Opt.Nofrun = 5;
      % run simulation
      SimResults = Model.Sim_Sensitivity_Analysis(OptTable,Opt);
      figure('Name','Sensitivity Analysis');
      SimVaryPlot(SimResults, 'F' ,'F' );